In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive/CamCAN

/content/drive/My Drive/CamCAN


In [4]:
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import os
from random import sample

from keras.layers import Dense, Conv3D, Flatten, Dropout, Conv3DTranspose, LeakyReLU, Input, Concatenate, Subtract, BatchNormalization, ZeroPadding3D, Cropping3D
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential, load_model, Model
from sklearn.model_selection import train_test_split

In [5]:
dtis = 0
for individual in os.listdir('data/dwi_coreg'):
    try:
        print(individual)
        loaded = nb.load('data/dwi_coreg/' +individual +'/dwi/dti_FA_ants.nii.gz').get_fdata()      
        loaded = loaded[10:170, 10:210, 0:170]
        pooled_loaded = loaded  
        if not dtis is 0:
          dtis = np.concatenate([np.expand_dims(pooled_loaded, axis=0), dtis], axis=0)
        else:
          dtis = np.expand_dims(pooled_loaded, axis=0)
        del loaded
    except FileNotFoundError:
        print('Error: ' + individual)

sub-CC110098
sub-CC120319
sub-CC120313
sub-CC120550
sub-CC120640
sub-CC220697
sub-CC121685
sub-CC210148
sub-CC220843
sub-CC220419
sub-CC220806
sub-CC220920
sub-CC220526
sub-CC220535
sub-CC121158
sub-CC221002
sub-CC221054
sub-CC320359
sub-CC222797
sub-CC310052
sub-CC310256
sub-CC310450
sub-CC320202
sub-CC221886
sub-CC221755
sub-CC223085
sub-CC320417
sub-CC320574
Error: sub-CC320574
sub-CC410040
sub-CC320680
Error: sub-CC320680
sub-CC322186
Error: sub-CC322186
sub-CC410220
sub-CC410182
sub-CC321291
Error: sub-CC321291
sub-CC320621
Error: sub-CC320621
sub-CC410129
sub-CC320888
Error: sub-CC320888
sub-CC420094
sub-CC510076
sub-CC420100
sub-CC420566
sub-CC420222
sub-CC420582
sub-CC510039
sub-CC510115
sub-CC420231
sub-CC420180
Error: sub-CC420180
sub-CC510237
sub-CC510259
sub-CC510323
sub-CC520197
sub-CC510648
sub-CC510354
sub-CC510329
sub-CC520136
sub-CC510483
sub-CC520287
sub-CC510474
sub-CC520055
sub-CC520377
sub-CC620262
sub-CC610671
sub-CC520775
sub-CC520562
sub-CC520624
sub-CC620499
su

In [6]:
#print(t1s.shape)
dtis = np.expand_dims(dtis, -1)
print(dtis.shape)

(80, 160, 200, 170, 1)


In [7]:

def PSNR(y_true, y_pred):
    max_pixel = 1.0
    return -(10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1))))

def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

def MSSSIM(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim_multiscale(
    y_true, y_pred, 1, filter_size=6,
    filter_sigma=1.5, k1=0.01, k2=0.03))

def L1(y_true, y_pred):
  return K.abs(y_true-y_pred)
autoencoder = Sequential()
autoencoder.add(Conv3D(filters=64, kernel_size=5, strides = 3, activation='relu', padding='same', input_shape = (160, 200, 170, 1)))#(t1s.shape[1], t1s.shape[2], t1s.shape[3], t1s.shape[4])))
#autoencoder.add(Conv3D(filters=64, kernel_size=3, strides = 2, padding='same', activation = 'relu'))
autoencoder.add(Conv3D(filters=1, kernel_size=3, padding='same', activation='relu')) 
autoencoder.add(Conv3DTranspose(filters=64, kernel_size=5, strides=3, padding='same', activation='relu'))
autoencoder.add(Cropping3D(cropping=((1,1), (0,1), (0, 1))))

#autoencoder.add(Conv3DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'))
autoencoder.add(Conv3D(filters=1, kernel_size=2, padding='same', activation = 'sigmoid'))

#autoencoder.add(ZeroPadding3D(padding=(0,0,0)))
#autoencoder.add(Cropping3D(cropping=((1,1), (0,1), (0, 1))))

autoencoder.compile(optimizer='adam',loss=MSSSIM)


In [8]:
train_t1s, test_t1s = train_test_split(dtis, test_size=0.1)

In [9]:

for i in range(30):
  for _ in range(20):
    chosen_indices= np.asarray(sample(list(range(len(train_t1s))), 1))
    autoencoder.fit(train_t1s[chosen_indices], train_t1s[chosen_indices])
  plt.imshow(autoencoder.predict(train_t1s[0:1])[0, 40, :, :, 0], cmap='Greys')
  plt.show()
  plt.imshow(autoencoder.predict(test_t1s[0:1])[0, 40, :, :, 0], cmap='Greys')
  plt.show()
  plt.imshow(train_t1s[0, 40, :, :, 0], cmap='Greys')
  plt.show()
  plt.imshow(test_t1s[0, 40, :, :, 0], cmap='Greys')
  plt.show()
  if i % 5 == 0:
    autoencoder.save('models/9_4_autoencoder_dti.h5')

Output hidden; open in https://colab.research.google.com to view.

In [10]:

for i, individual in enumerate(os.listdir('data/dwi_coreg')[:10]):
  try:
    dti_loaded = nb.load('data/dwi_coreg/' +individual +'/dwi/dti_FA_ants.nii.gz')

    dti_image = dti_loaded.get_fdata()[10:170, 10:210, 0:170]

    autoencoded_dti = autoencoder.predict(np.expand_dims(np.expand_dims(dti_image, 0), -1))
    ni_img = nb.Nifti1Image(dti_image, dti_loaded.affine)
    nb.save(ni_img, 'slide_making/' + individual + '_original_dti.nii.gz')

    ni_img = nb.Nifti1Image(autoencoded_dti[0, :, :, :, 0], dti_loaded.affine)
    nb.save(ni_img, 'slide_making/' + individual + '_autoencoded_dti.nii.gz')
  except FileNotFoundError:
    print('Error in ' + individual)
